# IBM & DataStax Demo - Banking AI Agent

## Before You Start

Make sure to check the prerequisites in the **README.md** file and run the previous sections.

## Part III - Agents and NoSQL

1. **Create a CQL table** to store banking transactions.
2. **Load sample data** into the table for testing.
3. **Connect to Astra** using the Data API.
4. **Create a Banking Agent Flow** on Langflow to automate processes.
5. **Connect Astra DB tools** to the agent for data interaction.
6. **Run the Flow** through the Langflow API to execute the automation.


# Astra DB NoSQL Tables

In the previous steps, we focused on the **Collection** data model — a semi-structured format that allows customers to store and query flexible JSON documents.

However, for use cases requiring very low latency and high throughput, **tables** should be the preferred data model.

In the upcoming steps, we will:

1. Create a **table** in Astra DB.
2. Insert data into the table using **CQL** (Cassandra Query Language).
3. Use the **Data-API** and **Astrapy** (Python client library) to interact with the table.


# Astra DB - Banking Use Case with NoSQL

In this section, we will generate a **banking statement** with transactions. To achieve this, we'll create a **table** in the same database from the previous steps and load some sample data.

## Astra DB - Driver Connection with CQL

To begin, you’ll need to download the [Secure Connect Bundle (SCB)](https://docs.datastax.com/en/astra-db-serverless/databases/secure-connect-bundle.html) from the Astra DB Dashboard.

<img src="./img/scb.png" alt="Secure Connect Bundle" width="600"/>

Click the **"Download SCB"** button, then copy the cURL command:

<img src="./img/scb1.png" alt="Secure Connect Bundle" width="600"/>

In [ ]:
# Run the curl command on the line below (add the exclamation mark at the start of the line) to download the secure-connect bundle for the Astra DB instance.
# This bundle is used to connect to the Astra DB instance from the Python code.
# The bundle contains the necessary certificates and configuration files.
!curl -o secure-connect-astra-ibm-demo.zip '<your secure-connect bundle URL>'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12323  100 12323    0     0  24218      0 --:--:-- --:--:-- --:--:-- 24210


You should now see the `secure-connect-astra-ibm-demo.zip` file in this folder.

With the file downloaded and your environment variables set, let's proceed to connect to the database.

In [1]:
# Import necessary libraries
from cassandra.query import SimpleStatement, PreparedStatement, BatchStatement
from cassandra.cluster import Cluster, PlainTextAuthProvider

In [3]:
# Make sure the environment variables are set
import os
from dotenv import load_dotenv
load_dotenv(override=True)

if os.getenv("ASTRA_DB_APPLICATION_TOKEN") is None:
    raise ValueError("Environment variable ASTRA_DB_APPLICATION_TOKEN not set")

if os.getenv("ASTRA_DB_APPLICATION_TOKEN")[:8] != "AstraCS:":
    raise ValueError(
        "Environment variable ASTRA_DB_APPLICATION_TOKEN invalid format")

if ".apps.astra.datastax.com" not in os.getenv("ASTRA_DB_API_ENDPOINT"):
    raise ValueError("Environment variable ASTRA_DB_API_ENDPOINT invalid")

print(
    f'Astra Token: {os.getenv("ASTRA_DB_APPLICATION_TOKEN")[:10]}...{os.getenv("ASTRA_DB_APPLICATION_TOKEN")[-5:]}')
print(f'Astra Endpoint: ...{os.getenv("ASTRA_DB_API_ENDPOINT")[-25:]}')
print("Good to go!")

python-dotenv could not parse statement starting at line 8


Astra Token: AstraCS:bf...35ec6
Astra Endpoint: ...2.apps.astra.datastax.com
Good to go!


In [4]:
# create a connection to the Astra DB
cluster = Cluster(
    cloud={
        "secure_connect_bundle": "./secure-connect-astra-ibm-demo.zip",
    },
    auth_provider=PlainTextAuthProvider(
        "token",
        os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    ),
)
session = cluster.connect("default_keyspace")

# Execute a simple query to test the connection
rs = session.execute("SELECT * FROM system_schema.tables where keyspace_name = 'default_keyspace' limit 1")
print(rs.one())

Row(keyspace_name='default_keyspace', table_name='banking_collection', additional_write_policy='99p', bloom_filter_fp_chance=0.01, caching=OrderedMapSerializedKey([('keys', 'ALL'), ('rows_per_partition', 'NONE')]), cdc=None, comment='{"collection":{"name":"banking_collection","schema_version":1,"options":{"vector":{"dimension":1024,"metric":"dot_product","sourceModel":"OTHER","service":{"provider":"nvidia","modelName":"NV-Embed-QA"}},"defaultId":{"type":""},"lexical":{"enabled":true,"analyzer":"standard"},"rerank":{"enabled":true,"service":{"provider":"nvidia","modelName":"nvidia/llama-3.2-nv-rerankqa-1b-v2","authentication":null,"parameters":null}}}}}', compaction=OrderedMapSerializedKey([('class', 'org.apache.cassandra.db.compaction.UnifiedCompactionStrategy')]), compression=OrderedMapSerializedKey([('chunk_length_in_kb', '16'), ('class', 'org.apache.cassandra.io.compress.LZ4Compressor')]), crc_check_chance=1.0, dclocal_read_repair_chance=0.0, default_time_to_live=0, extensions=Order

Before creating the table, let's review a brief summary of **Cassandra Data Modeling**:

<img src="./img/data_modeling.png" alt="Data Modeling" width="600"/>


In [5]:
# Lets create a table to store transactions by month

create_table = """
CREATE TABLE IF NOT EXISTS transactions_by_month (
    account_id UUID,
    year_month TEXT,                -- Format: 'YYYYMM', e.g., '202504'
    transaction_timestamp TIMESTAMP,
    transaction_id UUID,
    amount DECIMAL,
    currency TEXT,
    channel TEXT,                  -- e.g., 'app', 'credit_card'
    location TEXT,                 -- e.g., 'Geo: -23.5,-46.6'
    merchant TEXT,
    description TEXT,
    balance_after DECIMAL,         -- NEW: balance after this transaction

    PRIMARY KEY ((account_id, year_month), transaction_timestamp, transaction_id)
) WITH CLUSTERING ORDER BY (transaction_timestamp DESC, transaction_id ASC);

"""

session.execute(create_table)
print("Table transactions_by_month created successfully.")

Table transactions_by_month created successfully.


Note the **PRIMARY KEY** definition:

```sql
PRIMARY KEY ((account_id, year_month), transaction_timestamp, transaction_id)
```

- The combined **account_id** and **year_month** form the **partition key**.
- The **transaction_timestamp** and **transaction_id** are the **clustering keys**.

This design allows us to efficiently query transactions based on **account_id** and **year_month**, and then sort the results by **transaction_timestamp** and **transaction_id**.

### Astra DB CQL Console

The **Astra DB CQL Console** is a web-based tool that allows you to interact with the database using **CQL** (Cassandra Query Language). It provides an intuitive interface for executing queries, managing tables, and monitoring your database.

To access the CQL Console, navigate to the **Astra DB Dashboard** and click on the **"CQL Console"** button.

<img src="./img/console.png" alt="Console" width="600"/>

Use the **DESCRIBE** command to view the schema of a table:

```sql
USE default_keyspace;
DESCRIBE transactions_by_month;
```

<img src="./img/console1.png" alt="Console" width="600"/>

Now, let's proceed to load some data into the table.

In [6]:
# Run this cell to generate transactions
import uuid
import random
from datetime import datetime, timedelta
from faker import Faker
import random

fake = Faker()
start_date = datetime(2025, 6, 1)
end_date = datetime(2025, 7, 1)
# Define possible channels and categories
channels = ['mobile_app', 'web_banking', 'atm', 'branch', 'credit_card']
categories = ['groceries', 'dining', 'transportation',
              'entertainment', 'shopping', 'utilities', 'healthcare', 'travel']

# Prepare the insert statement
cmd = session.prepare("""INSERT INTO transactions_by_month (
    account_id, 
    year_month, 
    transaction_timestamp, 
    transaction_id, 
    amount, 
    currency, 
    channel, 
    location, 
    merchant, 
    description, 
    balance_after) VALUES (
    :account_id, 
    :year_month, 
    :transaction_timestamp, 
    :transaction_id, 
    :amount, 
    :currency, 
    :channel, 
    :location, 
    :merchant, 
    :description, 
    :balance_after)""")

# Prepare the batch statement
batch = BatchStatement()

# Generate 50 transactions for each customer
batch = BatchStatement()
customers = [uuid.uuid4() for _ in range(10)]
merchants = ['Walmart', 'Target', 'Amazon', 'Starbucks', 'McDonalds', 'Uber', 'Lyft', 'Netflix', 'Spotify', 'Apple Store']

for customer in customers:
    account_id = uuid.uuid4() 
    year_month = '202506'  # June 2025
    balance = random.uniform(1000.00, 10000.00)  # Starting balance
    current_date = start_date
    time_increment = (end_date - start_date) / 60  # Divide the month into 50 equal intervals
    
    
    for _ in range(50):
        transaction_id = uuid.uuid4()
        transaction_timestamp = current_date
        current_date += time_increment
        amount = round(random.uniform(-500.00, 500.00), 2)
        balance += amount
        merchant = random.choice(merchants)
        
        batch.add(cmd, {
            'account_id': account_id,
            'year_month': year_month,
            'transaction_timestamp': transaction_timestamp,
            'transaction_id': transaction_id,
            'amount': amount,
            'currency': 'USD',
            'channel': random.choice(channels),
            'location': 'Geo: -23.5,-46.6',
            'merchant': random.choice(merchants),
            'description': f"{random.choice(merchants)} - {random.choice(categories)}",
            'balance_after': balance
        })

    # Execute the batch
    session.execute(batch)
    print(f"Successfully inserted {len(batch) } transactions for {account_id} customer")
    batch.clear()
    
   

Successfully inserted 50 transactions for 6d62b152-c584-470e-b855-7e5677e7ba43 customer
Successfully inserted 50 transactions for b46750de-67ab-4275-a0a0-595ad79cdbcd customer
Successfully inserted 50 transactions for 970b6e98-80aa-4b22-9b3c-6b22e3b065b2 customer
Successfully inserted 50 transactions for d5289659-48c6-4483-adc6-2dbf4db43150 customer
Successfully inserted 50 transactions for 740a842e-d4ba-4a8b-bcce-5885237d25be customer
Successfully inserted 50 transactions for 1d877cec-cafa-4d29-aa8e-b8b906b2e63b customer
Successfully inserted 50 transactions for 9bd0c360-7fff-4552-83a8-25bb0cd99928 customer
Successfully inserted 50 transactions for 5b739d88-3cf4-40a1-8bd3-230b8b6b3248 customer
Successfully inserted 50 transactions for 5784d479-278c-4804-9223-b71daeb80f7a customer
Successfully inserted 50 transactions for fe29903d-b3d5-470e-baa6-ecec346c375e customer


Check the result on the CQL Console.

```
SELECT * FROM default_keyspace.transactions_by_month LIMIT 1;
```

<img src="./img/console2.png" alt="Console" width="600"/>

## Astra DB Data API for NoSQL Tables

You can access Astra DB **NoSQL tables** through the [Data API](https://docs.datastax.com/en/astra-db-serverless/api-reference/dataapiclient.html), a REST API service provided by Astra DB.

This connection method is recommended for developers who prefer simplifying app development by avoiding **CQL commands** and abstracting the complexities of using database drivers.

For more information, refer to the [Data API Tables documentation](https://docs.datastax.com/en/astra-db-serverless/api-reference/tables.html).

The Data API also supports clients for several languages:

- **Python**: [Astrapy](https://github.com/datastax/astrapy)
- **TypeScript**: [astra-db-ts](https://github.com/datastax/astra-db-ts)
- **Java**: [astra-db-java](https://github.com/datastax/astra-db-java)

Other languages can use HTTP commands to interact with the Data API. For examples, check the [Postman collection](https://www.postman.com/datastax/stargate-cassandra/collection/knuwhcu/data-api-vector-collection?tab=overview).

### Astrapy

**Astrapy** is the Python client we will use in the next section.

Astrapy can be used to establish a connection with Astra DB, create objects (like tables, indexes, and collections), and interact with tables and collections.

Since we've already created a table, let’s proceed by connecting to Astra and querying the table.

In [7]:
from astrapy import DataAPIClient
client = DataAPIClient()
astra_db = client.get_database(os.getenv("ASTRA_DB_API_ENDPOINT"), 
                               token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
                               keyspace="default_keyspace")

With the connection established, we can now create a reference to the table and execute queries on it.


In [8]:
transactions_table = astra_db.get_table( "transactions_by_month")
rs = transactions_table.find({"account_id": "0a662701-cb5d-4253-834c-b28233a832fa",
                        "year_month": "202506"},
                             limit=3)

for row in rs:
    print(row)

{'account_id': UUID('0a662701-cb5d-4253-834c-b28233a832fa'), 'year_month': '202506', 'transaction_timestamp': DataAPITimestamp(timestamp_ms=1750852800000 [2025-06-25T12:00:00.000Z]), 'transaction_id': UUID('8eea0b07-6e58-4a7b-ae74-86b518918168'), 'amount': Decimal('394.16000000000002501110429875552654266357421875'), 'balance_after': Decimal('6458.32253184610817697830498218536376953125'), 'channel': 'credit_card', 'currency': 'USD', 'description': 'Lyft - groceries', 'location': 'Geo: -23.5,-46.6', 'merchant': 'Netflix'}
{'account_id': UUID('0a662701-cb5d-4253-834c-b28233a832fa'), 'year_month': '202506', 'transaction_timestamp': DataAPITimestamp(timestamp_ms=1750809600000 [2025-06-25T00:00:00.000Z]), 'transaction_id': UUID('82311281-8893-4951-9feb-07266821a63f'), 'amount': Decimal('330.05000000000001136868377216160297393798828125'), 'balance_after': Decimal('6064.1625318461083224974572658538818359375'), 'channel': 'branch', 'currency': 'USD', 'description': 'Spotify - entertainment', 'l

# Langflow Agent

In this section, we will create an **AI Agent** that accesses the table with banking transactions.

The flow is available in this repository (file: **Part_III - Langflow Agent with NoSQL data from Astra DB.json**).

### Creating a Blank Flow

After accessing Langflow, create a new blank flow:

<img src="./img/lf_blank_flow.png" alt="Console" width="600"/>

Next, drag the **JSON** file onto the canvas or use the **Import** option from the menu:

<img src="./img/lf_import.png" alt="Console" width="600"/>

Once the flow is imported, it should look like this:

<img src="./img/lf_agent.png" alt="Console" width="600"/>

## Customizing the Flow

### Text Input

To customize the flow, start by changing the content of the **Text Input** component to an **account_id** stored in your database. Execute the next cell to retrieve this value:


In [44]:
rs = session.execute("SELECT account_id FROM default_keyspace.transactions_by_month limit 1")
print(rs.one())

Row(account_id=UUID('c45b4f5f-5c7b-41b3-b969-5cf53476e2f7'))


Click on the **edit** button and input the **account_id**.

<img src="./img/lf_agent_edit_acc.png" alt="Console" width="600"/>

### Astra DB CQL

In the **"Astra DB CQL"** component, click on the options for **"Astra DB Application Token"** and select **"+ Add New Variable"**:

<img src="./img/lf_agent_var.png" alt="Console" width="300"/>

Next, create a variable named **"ASTRA_IBM_TOKEN"** and set its value to the **ASTRA_DB_APPLICATION_TOKEN** found in your `.env` file. Save the variable.

<img src="./img/lf_agent_var2.png" alt="Console" width="300"/>

Do the same for the **API Endpoint**. Create a variable named **"ASTRA_IBM_API"** and set its value to **ASTRA_DB_API_ENDPOINT** from your `.env` file, then save it.

Now, click on the **"Tool Parameters"** button. These are the parameters the **LLM** will fill in order to run the tool.

<img src="./img/lf_agent_tool_param.png" alt="Console" width="600"/>

### Agent

Since we are using the OpenAI model to run our agent, you need to define the **OpenAI API Key** the agent will use.

Follow the same procedure to create a variable named **"OPENAI_API_KEY"**, save your key, and select it in the component.

## Running the Flow

Click on the **"Playground"** button and enter the question:

> How much did I spend between June 15 and 25, 2025?

Your result should look something like this (though the numbers and dates may vary):

<img src="./img/lf_agent_run1.png" alt="Console" width="600"/>

Next, click on the **"detail"** button to inspect the execution.

The **"Input"** section shows how the LLM filled the tool parameters:

<img src="./img/lf_agent_run2.png" alt="Console" width="600"/>

The **"Output"** section shows the data returned from the tool (Astra DB) to the LLM:

<img src="./img/lf_agent_run3.png" alt="Console" width="600"/>

Finally, the LLM has real-time context to answer the question:

<img src="./img/lf_agent_run4.png" alt="Console" width="600"/>

# Recap

In this exercise, you learned:

**Using Astra:**
- The key features available for companies
- Common use cases for Astra DB
- How to connect to Astra DB
- How to create tables within Astra
- How to use the Astra DB Console effectively
- How to load data into Astra DB

**Using Langflow:**
- How to import and manage flows
- How to customize the execution of flows
- How to configure the Astra DB Tool within Langflow
- How to create and reuse variables in flows
- How to inspect and analyze agent execution